#![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png) + ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)
#Power Plant Machine Learning Pipeline Application
This notebook is an end-to-end exercise of performing Extract-Transform-Load and Exploratory Data Analysis on a real-world dataset, and then applying several different machine learning algorithms to solve a supervised regression problem on the dataset.

** This notebook covers: **
* *Part 1: Business Understanding*
* *Part 2: Load Your Data*
* *Part 3: Explore Your Data*
* *Part 4: Visualize Your Data*
* *Part 5: Data Preparation*
* *Part 6: Data Modeling*
* *Part 7: Tuning and Evaluation*

*Our goal is to accurately predict power output given a set of environmental readings from various sensors in a natural gas-fired power generation plant.*


** Background **

Power generation is a complex process, and understanding and predicting power output is an important element in managing a plant and its connection to the power grid. The operators of a regional power grid create predictions of power demand based on historical information and environmental factors (e.g., temperature). They then compare the predictions against available resources (e.g., coal, natural gas, nuclear, solar, wind, hydro power plants). Power generation technologies such as solar and wind are highly dependent on environmental conditions, and all generation technologies are subject to planned and unplanned maintenance.

Here is an real-world example of predicted demand (on two time scales), actual demand, and available resources from the California power grid: http://www.caiso.com/Pages/TodaysOutlook.aspx

![](http://content.caiso.com/outlook/SP/ems_small.gif)

The challenge for a power grid operator is how to handle a shortfall in available resources versus actual demand. There are three solutions to  a power shortfall: build more base load power plants (this process can take many years to decades of planning and construction), buy and import power from other regional power grids (this choice can be very expensive and is limited by the power transmission interconnects between grids and the excess power available from other grids), or turn on small [Peaker or Peaking Power Plants](https://en.wikipedia.org/wiki/Peaking_power_plant). Because grid operators need to respond quickly to a power shortfall to avoid a power outage, grid operators rely on a combination of the last two choices. In this exercise, we'll focus on the last choice.

** The Business Problem **

Because they supply power only occasionally, the power supplied by a peaker power plant commands a much higher price per kilowatt hour than power from a power grid's base power plants. A peaker plant may operate many hours a day, or it may operate only a few hours per year, depending on the condition of the region's electrical grid. Because of the cost of building an efficient power plant, if a peaker plant is only going to be run for a short or highly variable time it does not make economic sense to make it as efficient as a base load power plant. In addition, the equipment and fuels used in base load plants are often unsuitable for use in peaker plants because the fluctuating conditions would severely strain the equipment.

The power output of a peaker power plant varies depending on environmental conditions, so the business problem is _predicting the power output of a peaker power plant as a function of the environmental conditions_ -- since this would enable the grid operator to make economic tradeoffs about the number of peaker plants to turn on (or whether to buy expensive power from another grid).

Given this business problem, we need to first perform Exploratory Data Analysis to understand the data and then translate the business problem (predicting power output as a function of envionmental conditions) into a Machine Learning task.  In this instance, the ML task is regression since the label (or target) we are trying to predict is numeric. We will use an [Apache Spark ML Pipeline](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark-ml-package) to perform the regression.

The real-world data we are using in this notebook consists of 9,568 data points, each with 4 environmental attributes collected from a Combined Cycle Power Plant over 6 years (2006-2011), and is provided by the University of California, Irvine at [UCI Machine Learning Repository Combined Cycle Power Plant Data Set](https://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant). You can find more details about the dataset on the UCI page, including the following background publications:
* Pinar Tüfekci, [Prediction of full load electrical power output of a base load operated combined cycle power plant using machine learning methods](http://www.journals.elsevier.com/international-journal-of-electrical-power-and-energy-systems/), International Journal of Electrical Power & Energy Systems, Volume 60, September 2014, Pages 126-140, ISSN 0142-0615.
* Heysem Kaya, Pinar Tüfekci and Fikret S. Gürgen: [Local and Global Learning Methods for Predicting Power of a Combined Gas & Steam Turbine](http://www.cmpe.boun.edu.tr/~kaya/kaya2012gasturbine.pdf), Proceedings of the International Conference on Emerging Trends in Computer and Electronics Engineering ICETCEE 2012, pp. 13-18 (Mar. 2012, Dubai).

**TODO**: Read the documentation and examples for [Spark Machine Learning Pipeline](https://spark.apache.org/docs/1.6.2/ml-guide.html#main-concepts-in-pipelines).

## Part 1: Business Understanding
The first step in any machine learning task is to understand the business need.

As described in the overview we are trying to predict power output given a set of readings from various sensors in a gas-fired power generation plant.

The problem is a regression problem since the label (or target) we are trying to predict is numeric.

## Part 2: Extract-Transform-Load (ETL) Your Data

Now that we understand what we are trying to do, the first step is to load our data into a format we can query and use.  This is known as ETL or "Extract-Transform-Load".  We will load our file from Amazon S3.

Note: Alternatively we could upload our data using "Databricks Menu > Tables > Create Table", assuming we had the raw files on our local computer.

Our data is available on Amazon s3 at the following path:

`dbfs:/databricks-datasets/power-plant/data`


**TODO**: Let's start by printing a sample of the data.

We'll use the built-in Databricks functions for exploring the Databricks filesystem (DBFS)

Use `display(dbutils.fs.ls("/databricks-datasets/power-plant/data"))` to list the files in the directory

In [4]:
display(dbutils.fs.ls("/databricks-datasets/power-plant/data"))

path,name,size
dbfs:/databricks-datasets/power-plant/data/Sheet1.tsv,Sheet1.tsv,308693
dbfs:/databricks-datasets/power-plant/data/Sheet2.tsv,Sheet2.tsv,308693
dbfs:/databricks-datasets/power-plant/data/Sheet3.tsv,Sheet3.tsv,308693
dbfs:/databricks-datasets/power-plant/data/Sheet4.tsv,Sheet4.tsv,308693
dbfs:/databricks-datasets/power-plant/data/Sheet5.tsv,Sheet5.tsv,308693


Next, use the `dbutils.fs.head` command to look at the first 65,536 bytes of the first file in the directory.

Use `print dbutils.fs.head("/databricks-datasets/power-plant/data/Sheet1.tsv")` to list the files in the directory

In [6]:
print dbutils.fs.head("/databricks-datasets/power-plant/data/Sheet1.tsv")

[Truncated to first 65536 bytes]
AT	V	AP	RH	PE
14.96	41.76	1024.07	73.17	463.26
25.18	62.96	1020.04	59.08	444.37
5.11	39.4	1012.16	92.14	488.56
20.86	57.32	1010.24	76.64	446.48
10.82	37.5	1009.23	96.62	473.9
26.27	59.44	1012.23	58.77	443.67
15.89	43.96	1014.02	75.24	467.35
9.48	44.71	1019.12	66.43	478.42
14.64	45	1021.78	41.25	475.98
11.74	43.56	1015.14	70.72	477.5
17.99	43.72	1008.64	75.04	453.02
20.14	46.93	1014.66	64.22	453.99
24.34	73.5	1011.31	84.15	440.29
25.71	58.59	1012.77	61.83	451.28
26.19	69.34	1009.48	87.59	433.99
21.42	43.79	1015.76	43.08	462.19
18.21	45	1022.86	48.84	467.54
11.04	41.74	1022.6	77.51	477.2
14.45	52.75	1023.97	63.59	459.85
13.97	38.47	1015.15	55.28	464.3
17.76	42.42	1009.09	66.26	468.27
5.41	40.07	1019.16	64.77	495.24
7.76	42.28	1008.52	83.31	483.8
27.23	63.9	1014.3	47.19	443.61
27.36	48.6	1003.18	54.93	436.06
27.47	70.72	1009.97	74.62	443.25
14.6	39.31	1011.11	72.52	464.16
7.91	39.96	1023.57	88.44	475.52
5.81	35.79	1012.14	92.28	484.41
30.53	65.18	1012.69	41.85	437.89
23.87	63.94	1019.02	44.28	445.11
26.09	58.41	1013.64	64.58	438.86
29.27	66.85	1011.11	63.25	440.98
27.38	74.16	1010.08	78.61	436.65
24.81	63.94	1018.76	44.51	444.26
12.75	44.03	1007.29	89.46	465.86
24.66	63.73	1011.4	74.52	444.37
16.38	47.45	1010.08	88.86	450.69
13.91	39.35	1014.69	75.51	469.02
23.18	51.3	1012.04	78.64	448.86
22.47	47.45	1007.62	76.65	447.14
13.39	44.85	1017.24	80.44	469.18
9.28	41.54	1018.33	79.89	482.8
11.82	42.86	1014.12	88.28	476.7
10.27	40.64	1020.63	84.6	474.99
22.92	63.94	1019.28	42.69	444.22
16	37.87	1020.24	78.41	461.33
21.22	43.43	1010.96	61.07	448.06
13.46	44.71	1014.51	50	474.6
9.39	40.11	1029.14	77.29	473.05
31.07	73.5	1010.58	43.66	432.06
12.82	38.62	1018.71	83.8	467.41
32.57	78.92	1011.6	66.47	430.12
8.11	42.18	1014.82	93.09	473.62
13.92	39.39	1012.94	80.52	471.81
23.04	59.43	1010.23	68.99	442.99
27.31	64.44	1014.65	57.27	442.77
5.91	39.33	1010.18	95.53	491.49
25.26	61.08	1013.68	71.72	447.46
27.97	58.84	1002.25	57.88	446.11
26.08	52.3	1007.03	63.34	442.44
29.01	65.71	1013.61	48.07	446.22
12.18	40.1	1016.67	91.87	471.49
13.76	45.87	1008.89	87.27	463.5
25.5	58.79	1016.02	64.4	440.01
28.26	65.34	1014.56	43.4	441.03
21.39	62.96	1019.49	72.24	452.68
7.26	40.69	1020.43	90.22	474.91
10.54	34.03	1018.71	74	478.77
27.71	74.34	998.14	71.85	434.2
23.11	68.3	1017.83	86.62	437.91
7.51	41.01	1024.61	97.41	477.61
26.46	74.67	1016.65	84.44	431.65
29.34	74.34	998.58	81.55	430.57
10.32	42.28	1008.82	75.66	481.09
22.74	61.02	1009.56	79.41	445.56
13.48	39.85	1012.71	58.91	475.74
25.52	69.75	1010.36	90.06	435.12
21.58	67.25	1017.39	79	446.15
27.66	76.86	1001.31	69.47	436.64
26.96	69.45	1013.89	51.47	436.69
12.29	42.18	1016.53	83.13	468.75
15.86	43.02	1012.18	40.33	466.6
13.87	45.08	1024.42	81.69	465.48
24.09	73.68	1014.93	94.55	441.34
20.45	69.45	1012.53	91.81	441.83
15.07	39.3	1019	63.62	464.7
32.72	69.75	1009.6	49.35	437.99
18.23	58.96	1015.55	69.61	459.12
35.56	68.94	1006.56	38.75	429.69
18.36	51.43	1010.57	90.17	459.8
26.35	64.05	1009.81	81.24	433.63
25.92	60.95	1014.62	48.46	442.84
8.01	41.66	1014.49	76.72	485.13
19.63	52.72	1025.09	51.16	459.12
20.02	67.32	1012.05	76.34	445.31
10.08	40.72	1022.7	67.3	480.8
27.23	66.48	1005.23	52.38	432.55
23.37	63.77	1013.42	76.44	443.86
18.74	59.21	1018.3	91.55	449.77
14.81	43.69	1017.19	71.9	470.71
23.1	51.3	1011.93	80.05	452.17
10.72	41.38	1021.6	63.77	478.29
29.46	71.94	1006.96	62.26	428.54
8.1	40.64	1020.66	89.04	478.27
27.29	62.66	1007.63	58.02	439.58
17.1	49.69	1005.53	81.82	457.32
11.49	44.2	1018.79	91.14	475.51
23.69	65.59	1010.85	88.92	439.66
13.51	40.89	1011.03	84.83	471.99
9.64	39.35	1015.1	91.76	479.81
25.65	78.92	1010.83	86.56	434.78
21.59	61.87	1011.18	57.21	446.58
27.98	58.33	1013.92	54.25	437.76
18.8	39.72	1001.24	63.8	459.36
18.28	44.71	1016.99	33.71	462.28
13.55	43.48	1016.08	67.25	464.33
22.99	46.21	1010.71	60.11	444.36
23.94	59.39	1014.32	74.55	438.64
13.74	34.03	1018.69	67.34	470.49
21.3	41.1	1001.86	42.75	455.13
27.54	66.93	1017.06	55.2	450.22
24.81	63.73	1009.

`dbutils.fs` has its own help facility, which we can use to see the various available functions.

In [8]:
dbutils.fs.help()

dbutils.fs provides utilities for working with FileSystems. Most methods in
this package can take either a DBFS path (e.g., "/foo" or "dbfs:/foo"), or
another FileSystem URI.

For more info about a method, use dbutils.fs.help("methodName") .

In notebooks, you can also use the %fs shorthand to access DBFS. The %fs shorthand maps
straightforwardly onto dbutils calls. For example, "%fs head --maxBytes=10000 /file/path"
translates into "dbutils.fs.head("/file/path", maxBytes = 10000)".
 fsutils cp(from: String, to: String, recurse: boolean = false): boolean -> Copies a file or directory, possibly across FileSystems head(file: String, maxBytes: int = 65536): String -> Returns up to the first 'maxBytes' bytes of the given file as a String encoded in UTF-8 ls(dir: String): Seq -> Lists the contents of a directory mkdirs(dir: String): boolean -> Creates the given directory if it does not exist, also creating any necessary parent directories mv(from: String, to: String, recurse: boolean = false): boolean -> Moves a file or directory, possibly across FileSystems put(file: String, contents: String, overwrite: boolean = false): boolean -> Writes the given String out to a file, encoded in UTF-8 rm(dir: String, recurse: boolean = false): boolean -> Removes a file or directory mount mount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Mounts the given source directory into DBFS at the given mount point mounts: Seq -> Displays information about what is mounted within DBFS refreshMounts: boolean -> Forces all machines in this cluster to refresh their mount cache, ensuring they receive the most recent information unmount(mountPoint: String): boolean -> Deletes a DBFS mount point

### Exercise 2(a)

Now, let's use PySpark instead to print the first 5 lines of the data.

*Hint*: First create an RDD from the data by using [`sc.textFile("dbfs:/databricks-datasets/power-plant/data")`](https://spark.apache.org/docs/1.6.2/api/python/pyspark.html#pyspark.SparkContext.textFile) to read the data into an RDD.

*Hint*: Then figure out how to use the RDD [`take()`](https://spark.apache.org/docs/1.6.2/api/python/pyspark.html#pyspark.RDD.take) method to extract the first 5 lines of the RDD and print each line.

In [10]:
# TODO: Load the data and print the first five lines.
textRDD = sc.textFile("dbfs:/databricks-datasets/power-plant/data")

for line in textRDD.take(5):
  print line
  

AT	V	AP	RH	PE
14.96	41.76	1024.07	73.17	463.26
25.18	62.96	1020.04	59.08	444.37
5.11	39.4	1012.16	92.14	488.56
20.86	57.32	1010.24	76.64	446.48

From our initial exploration of a sample of the data, we can make several observations for the ETL process:
  - The data is a set of .tsv (Tab Seperated Values) files (i.e., each row of the data is separated using tabs)
  - There is a header row, which is the name of the columns
  - It looks like the type of the data in each column is consistent (i.e., each column is of type double)

Our schema definition from UCI appears below:
- AT = Atmospheric Temperature in C
- V = Exhaust Vacuum Speed
- AP = Atmospheric Pressure
- RH = Relative Humidity
- PE = Power Output.  This is the value we are trying to predict given the measurements above.

We are ready to create a DataFrame from the TSV data. Spark does not have a native method for performing this operation, however we can use [spark-csv](https://spark-packages.org/package/databricks/spark-csv), a third-party package from [SparkPackages](https://spark-packages.org/). The documentation and source code for [spark-csv](https://spark-packages.org/package/databricks/spark-csv) can be found on [GitHub](https://github.com/databricks/spark-csv). The Python API can be found [here](https://github.com/databricks/spark-csv#python-api).

(**Note**: In Spark 2.0, the CSV package is built into the DataFrame API.)

To use the [spark-csv](https://spark-packages.org/package/databricks/spark-csv) package, we use the [sqlContext.read.format()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.format) method to specify the input data source format: `'com.databricks.spark.csv'`

We can provide the [spark-csv](https://spark-packages.org/package/databricks/spark-csv) package with options using the [options()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.options) method. The available options are listed in the GitHub documentation [here](https://github.com/databricks/spark-csv#features).

We will use the following three options:
- `delimiter='\t'` because our data is tab delimited
- `header='true'` because our data has a header row
- `inferschema='true'` because we believe that all of the data is double values, so the package can dynamically infer the type of each column. *Note that this will require two pass over the data.*


The last component of creating the DataFrame is to specify the location of the data source using the [load()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.load) method: `"/databricks-datasets/power-plant/data"`

Putting everything together, we will use an operation of the following form:

  `sqlContext.read.format().options().load()`

### Exercise 2(b)

**TODO**: Create a DataFrame from the data.

*Hint*: Use the above template and fill in each of the methods.

In [12]:
# TODO: Replace <FILL_IN> with the appropriate code.
plantDF = sqlContext.read.format('com.databricks.spark.csv')\
                         .options(delimiter='\t', header='true', inferschema='true')\
                         .load("/databricks-datasets/power-plant/data")


In [13]:
# TEST
from test_helper import Test
expected = set([(s, 'double') for s in ('AP', 'AT', 'PE', 'RH', 'V')])
Test.assertEquals(expected, set(powerPlantDF.dtypes), "Incorrect schema for powerPlantDF")

1 test passed.

Check the names and types of the columns using the [dtypes](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame.dtypes) method.

In [15]:
print plantDF.dtypes

[('AT', 'double'), ('V', 'double'), ('AP', 'double'), ('RH', 'double'), ('PE', 'double')]

We can examine the data using the display() method.

In [17]:
display(plantDF)

AT,V,AP,RH,PE
14.96,41.76,1024.07,73.17,463.26
25.18,62.96,1020.04,59.08,444.37
5.11,39.4,1012.16,92.14,488.56
20.86,57.32,1010.24,76.64,446.48
10.82,37.5,1009.23,96.62,473.9
26.27,59.44,1012.23,58.77,443.67
15.89,43.96,1014.02,75.24,467.35
9.48,44.71,1019.12,66.43,478.42
14.64,45.0,1021.78,41.25,475.98
11.74,43.56,1015.14,70.72,477.5


### Part 2: Alternative Method to Load your Data
Instead of having [spark-csv](https://spark-packages.org/package/databricks/spark-csv) infer the types of the columns, we can specify the schema as a [DataType](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.types.DataType), which is a list of [StructField](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.types.StructType).

You can find a list of types in the [pyspark.sql.types](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#module-pyspark.sql.types) module. For our data, we will use [DoubleType()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.types.DoubleType).

For example, to specify that a column's name and type, we use: `StructField(`_name_`,` _type_`, True)`. (The third parameter, `True`, signifies that the column is nullable.)

### Exercise 2(c)

Create a custom schema for the power plant data.

In [19]:
# TODO: Fill in the custom schema.
from pyspark.sql.types import *

# Custom Schema for Power Plant
customSchema = StructType([ \
    StructField("AT", DoubleType(), True), \
    StructField("V", DoubleType(), True), \
    StructField("AP", DoubleType(), True), \
    StructField("RH", DoubleType(), True), \
    StructField("PE", DoubleType(), True) \
                          ])

In [20]:
# TEST
Test.assertEquals(set([f.name for f in customSchema.fields]), set(['AT', 'V', 'AP', 'RH', 'PE']), 'Incorrect column names in schema.')
Test.assertEquals(set([f.dataType for f in customSchema.fields]), set([DoubleType(), DoubleType(), DoubleType(), DoubleType(), DoubleType()]), 'Incorrect column types in schema.')

1 test passed.
1 test passed.

### Exercise 2(d)

Now, let's use the schema to read the data. To do this, we will modify the earlier `sqlContext.read.format` step. We can specify the schema by:
- Adding `schema = customSchema` to the load method (use a comma and add it after the file name)
- Removing the `inferschema='true'`option because we are explicitly specifying the schema

In [22]:
# TODO: Use the schema you created above to load the data again.
altPlantDF = sqlContext.read.format('com.databricks.spark.csv')\
                            .options(delimiter='\t', header='true')\
                            .load("/databricks-datasets/power-plant/data", schema=customSchema)

In [23]:
# TEST
expected = set([(s, 'double') for s in ('AP', 'AT', 'PE', 'RH', 'V')])
Test.assertEquals(expected, set(altPlantDF.dtypes), "Incorrect schema for powerPlantDF")

1 test passed.

Note that no Spark jobs are launched this time. That is because we specified the schema, so the [spark-csv](https://spark-packages.org/package/databricks/spark-csv) package does not have to read the data to infer the schema. We can use the [dtypes](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame.dtypes) method to examine the names and types of the columns. They should be identical to the names and types of the columns that were earlier inferred from the data.

When you run the following cell, data would not be read.

In [25]:
print altPlantDF.dtypes

[('AT', 'double'), ('V', 'double'), ('AP', 'double'), ('RH', 'double'), ('PE', 'double')]

Now we can examine the data using the display() method. *Note that this operation will cause the data to be read and the DataFrame will be created.*

In [27]:
display(altPlantDF)

AT,V,AP,RH,PE
14.96,41.76,1024.07,73.17,463.26
25.18,62.96,1020.04,59.08,444.37
5.11,39.4,1012.16,92.14,488.56
20.86,57.32,1010.24,76.64,446.48
10.82,37.5,1009.23,96.62,473.9
26.27,59.44,1012.23,58.77,443.67
15.89,43.96,1014.02,75.24,467.35
9.48,44.71,1019.12,66.43,478.42
14.64,45.0,1021.78,41.25,475.98
11.74,43.56,1015.14,70.72,477.5


## Part 3: Explore Your Data
Now that your data is loaded, the next step is to explore it and perform some basic analysis and visualizations.

This is a step that you should always perform **before** trying to fit a model to the data, as this step will often lead to important insights about your data.

First, let's register our DataFrame as an SQL table named `plant_dataset`.  Because you may run this lab multiple times, we'll take the precaution of removing any existing tables first.

We can delete any existing `plant_dataset` SQL table using the SQL command: `DROP TABLE IF EXISTS plant_dataset` (we also need to to delete any Hive data associated with the table, which we can do with a Databricks file system operation).

Once any prior table is removed, we can register our DataFrame as a SQL table using [sqlContext.registerDataFrameAsTable()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.SQLContext.registerDataFrameAsTable).

### 3(a)

**TODO**: Execute the prepared code in the following cell.

In [30]:
sqlContext.sql("DROP TABLE IF EXISTS plant_dataset")
dbutils.fs.rm("dbfs:/user/hive/warehouse/plant_dataset", True)
sqlContext.registerDataFrameAsTable(plantDF, "plant_dataset")

Now that our DataFrame exists as a SQL table, we can explore it using SQL commands.

To execute SQL in a cell, we use the `%sql` operator. The following cell is an example of using SQL to query the rows of the SQL table.

**NOTE**: `%sql` is a Databricks-only command. It calls `sqlContext.sql()` and passes the results to the Databricks-only `display()` function. These two statements are equivalent:

`%sql SELECT * FROM plant_dataset`

`display(sqlContext.sql("SELECT * FROM plant_dataset"))`

### 3(b)

**TODO**: Execute the prepared code in the following cell.

In [32]:
%sql
-- We can use %sql to query the rows
SELECT * FROM plant_dataset

AT,V,AP,RH,PE
14.96,41.76,1024.07,73.17,463.26
25.18,62.96,1020.04,59.08,444.37
5.11,39.4,1012.16,92.14,488.56
20.86,57.32,1010.24,76.64,446.48
10.82,37.5,1009.23,96.62,473.9
26.27,59.44,1012.23,58.77,443.67
15.89,43.96,1014.02,75.24,467.35
9.48,44.71,1019.12,66.43,478.42
14.64,45.0,1021.78,41.25,475.98
11.74,43.56,1015.14,70.72,477.5


### 3(c)

Use the SQL `desc` command to describe the schema, by executing the following cell.

In [34]:
%sql
desc plant_dataset

col_name,data_type,comment
AT,double,null
V,double,null
AP,double,null
RH,double,null
PE,double,null


**Schema Definition**

Once again, here's our schema definition:

- AT = Atmospheric Temperature in C
- V = Exhaust Vacuum Speed
- AP = Atmospheric Pressure
- RH = Relative Humidity
- PE = Power Output

PE is our label or target. This is the value we are trying to predict given the measurements.

*Reference [UCI Machine Learning Repository Combined Cycle Power Plant Data Set](https://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant)*

Let's perform some basic statistical analyses of all the columns.

We can get the DataFrame associated with a SQL table by using the [sqlContext.table()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.table) method and passing in the name of the SQL table. Then, we can use the DataFrame [describe()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame.describe) method with no arguments to compute some basic statistics for each column like count, mean, max, min and standard deviation.

In [37]:
df = sqlContext.table("plant_dataset")

display(df.describe())

summary,AT,V,AP,RH,PE
count,47840,47840,47840,47840,47840
mean,19.651231187290996,54.30580372073594,1013.2590781772572,73.30897784280918,454.36500940635506
stddev,7.452161658340004,12.707361709685806,5.938535418520816,14.599658352081477,17.06628146683769
min,1.81,25.36,992.89,25.56,420.26
max,37.11,81.56,1033.3,100.16,495.76


##Part 4: Visualize Your Data

To understand our data, we will look for correlations between features and the label.  This can be important when choosing a model.  E.g., if features and a label are linearly correlated, a linear model like Linear Regression can do well; if the relationship is very non-linear, more complex models such as Decision Trees can be better. We can use Databrick's built in visualization to view each of our predictors in relation to the label column as a scatter plot to see the correlation between the predictors and the label.

### Exercise 4(a)

** Add figures to the following: **
Let's see if there is a corellation between Temperature and Power Output. We can use a SQL query to create a new table consisting of only the AtmosphericTemperature (AT) and PowerOutput (PE) columns, and then use a scatter plot with AtmosphericTemperature on the X axis and PowerOutput on the Y axis to visualize the relationship (if any) between AtmosphericTemperature and PowerOutput.


Perform the following steps:

- Run the following cell
- Click on the drop down next to the "Bar chart" icon a select "Scatter" to turn the table into a Scatter plot

<img src="http://spark-mooc.github.io/web-assets/images/cs110x/change-plot-type-scatter.png" style="border: 1px solid #999999"/>

- Click on "Plot Options..."
- In the Values box, click on "Temperature" and drag it before "Power", as shown below:

<img src="https://raw.githubusercontent.com/hkust-comp4651-2018f/assets/master/assignment-5/customize_plot_small.png" style="border: 1px solid #999999"/>

- Apply your changes by clicking the Apply button
- Increase the size of the graph by clicking and dragging the size control

In [39]:
%sql
select AT as AtmosphericTemperature, PE as PowerOutput from plant_dataset

AtmosphericTemperature,PowerOutput
14.96,463.26
25.18,444.37
5.11,488.56
20.86,446.48
10.82,473.9
26.27,443.67
15.89,467.35
9.48,478.42
14.64,475.98
11.74,477.5


It looks like there is strong linear correlation between Temperature and Power Output.

** ASIDE: A quick physics lesson**: This correlation is to be expected as the second law of thermodynamics puts a fundamental limit on the [thermal efficiency](https://en.wikipedia.org/wiki/Thermal_efficiency) of all heat-based engines. The limiting factors are:
 - The temperature at which the heat enters the engine \\( T_{H} \\)
 - The temperature of the environment into which the engine exhausts its waste heat \\( T_C \\)

Our temperature measurements are the temperature of the environment. From [Carnot's theorem](https://en.wikipedia.org/wiki/Carnot%27s_theorem_%28thermodynamics%29), no heat engine working between these two temperatures can exceed the Carnot Cycle efficiency:
\\[ n_{th} \le 1 - \frac{T_C}{T_H}  \\]

Note that as the environmental temperature increases, the efficiency decreases -- _this is the effect that we see in the above graph._

### Exercise 4(b)

Use SQL to create a scatter plot of PowerOutput(PE) as a function of VacuumSpeed (V).
Name the y-axis "PowerOutput" and the x-axis "VacuumSpeed"

In [42]:
%sql
select PE as PowerOutput, V as VacuumSpeed from plant_dataset

Power,ExhaustVacuum
463.26,41.76
444.37,62.96
488.56,39.4
446.48,57.32
473.9,37.5
443.67,59.44
467.35,43.96
478.42,44.71
475.98,45.0
477.5,43.56


Let's continue exploring the relationships (if any) between the variables and Power Output.

### Exercise 4(c)

Use SQL to create a scatter plot of PowerOutput(PE) as a function of AtmosphericPressure (AP).
Name the y-axis "PowerOutput" and the x-axis "AtmosphericPressure"

In [44]:
%sql
-- TODO: Replace <FILL_IN> with the appropriate SQL command.
select PE as PowerOutput, AP as AtmosphericPressure from plant_dataset

PowerOutput,AtmosphericPressure
463.26,1024.07
444.37,1020.04
488.56,1012.16
446.48,1010.24
473.9,1009.23
443.67,1012.23
467.35,1014.02
478.42,1019.12
475.98,1021.78
477.5,1015.14


### Exercise 4(d)

Use SQL to create a scatter plot of PowerOutput(PE) as a function of RelativeHumidity(RH).
Name the y-axis "PowerOutput" and the x-axis "RelativeHumidity"

In [46]:
%sql
-- TODO: Replace <FILL_IN> with the appropriate SQL command.
select PE as Power, RH as RelativeHumidity from plant_dataset

Power,RelativeHumidity
463.26,73.17
444.37,59.08
488.56,92.14
446.48,76.64
473.9,96.62
443.67,58.77
467.35,75.24
478.42,66.43
475.98,41.25
477.5,70.72


##Part 5: Data Preparation

The next step is to prepare the data for machine learning. Since all of this data is numeric and consistent this is a simple and straightforward task.

The goal is to use machine learning to determine a function that yields the output power as a function of a set of predictor features. The first step in building our ML pipeline is to convert the predictor features from DataFrame columns to Feature Vectors using the [pyspark.ml.feature.VectorAssembler()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.feature.VectorAssembler) method.

The VectorAssembler is a transformer that combines a given list of columns into a single vector column. It is useful for combining raw features and features generated by different feature transformers into a single feature vector, in order to train ML models like logistic regression and decision trees. VectorAssembler takes a list of input column names (each is a string) and the name of the output column (as a string).

### Exercise 5(a)

- Read the Spark documentation and useage examples for [VectorAssembler](https://spark.apache.org/docs/1.6.2/ml-features.html#vectorassembler).
- Convert the `plant_dataset` SQL table into a DataFrame.
- Set the vectorizer's input columns to a list of the four columns of the input DataFrame: `["AT", "V", "AP", "RH"]`.
- Set the vectorizer's output column name to `"features"`.

In [48]:
# TODO: Replace <FILL_IN> with the appropriate code
from pyspark.ml.feature import VectorAssembler

datasetDF = sqlContext.table('plant_dataset')

vectorizer = VectorAssembler()
vectorizer.setInputCols(["AT", "V", "AP", "RH"])
vectorizer.setOutputCol("features")

Out[ 18 ]: VectorAssembler_41deab30fec51ab9393a

In [49]:
# TEST
Test.assertEquals(set(vectorizer.getInputCols()), {"AT", "V", "AP", "RH"}, "Incorrect vectorizer input columns")
Test.assertEquals(vectorizer.getOutputCol(), "features", "Incorrect vectorizer output column")

1 test passed.
1 test passed.

##Part 6: Data Modeling
Now let's model our data to predict what the power output will be given a set of sensor readings

Our first model will be based on simple linear regression since we saw some linear patterns in our data based on the scatter plots during the exploration stage.

We need a way of evaluating how well our linear regression model predicts power output as a function of input parameters. We can do this by splitting up our initial data set into a _Training Set_ used to train our model and a _Test Set_ used to evaluate the model's performance in giving predictions. We can use a DataFrame's [randomSplit()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame.randomSplit) method to split our dataset. The method takes a list of weights and an optional random seed. The seed is used to initialize the random number generator used by the splitting function.

### Exercise 6(a)

Use the [randomSplit()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame.randomSplit) method to divide up `datasetDF` into a trainingSetDF (15% of the input DataFrame) and a testSetDF (85% of the input DataFrame), and for reproducibility, use the seed 1900009193L. Then cache each DataFrame in memory to maximize performance.

In [51]:
# TODO: Replace <FILL_IN> with the appropriate code.
# We'll hold out 15% of our data for testing and leave 85% for training
seed = 1900009193L
(split15DF, split85DF) = datasetDF.randomSplit([.15, .85], seed)

# Let's cache these datasets for performance
testSetDF = split15DF.cache()
trainingSetDF = split85DF.cache()

In [52]:
# TEST
Test.assertEquals(trainingSetDF.count(), 40674, "Incorrect size for training data set")
Test.assertEquals(testSetDF.count(), 7166, "Incorrect size for test data set")

1 test passed.
1 test passed.

Next we'll create a Linear Regression Model and use the built in help to identify how to train it. See API details for [Linear Regression](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.regression.LinearRegression) in the ML guide.

### Exercise 6(b)

- Read the documentation and examples for [Linear Regression](https://spark.apache.org/docs/1.6.2/ml-classification-regression.html#linear-regression)
- Run the next cell

In [54]:
# ***** LINEAR REGRESSION MODEL ****

from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import LinearRegressionModel
from pyspark.ml import Pipeline

# Let's initialize our linear regression learner
lr = LinearRegression()

# We use explain params to dump the parameters we can use
print(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
epsilon: The shape parameter to control the amount of robustness. Must be > 1.0. Only valid when loss is huber (default: 1.35)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
loss: The loss function to be optimized. Supported options: squaredError, huber. (default: squaredError)
maxIter: max number of iterations (>= 0). (default: 100)
predictionCol: prediction column name. (default: prediction)
regParam: regularization parameter (>= 0). (default: 0.0)
solver: The solver algorithm for optimization. Supported options: auto, normal, l-bfgs. (default: auto)
standardization: whether to standardize the training features before fitting the model. (default: True)
tol: the convergence tolerance for iterative algorithms (>= 0). (default: 1e-06)
weightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (undefined)

The cell below is based on the [Spark ML Pipeline API for Linear Regression](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.regression.LinearRegression).

The first step is to set the parameters for the method:
- Set the name of the prediction column to "Prediction_PE"
- Set the name of the label column to "PE"
- Set the maximum number of iterations to 100
- Set the regularization parameter to 0.15

Next, we create the [ML Pipeline](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.Pipeline) and set the stages to the Vectorizer and Linear Regression learner we created earlier.

Finally, we create a model by training on `trainingSetDF`.

### Exercise 6(c)

- Read the [Linear Regression](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.regression.LinearRegression) documentation
- Run the next cell, and be sure you understand what's going on.

In [56]:
# Now we set the parameters for the method
lr.setPredictionCol("Prediction_PE")\
  .setLabelCol("PE")\
  .setMaxIter(100)\
  .setRegParam(0.15)


# We will use the new spark.ml pipeline API. If you have worked with scikit-learn this will be very familiar.
lrPipeline = Pipeline()

lrPipeline.setStages([vectorizer, lr])

# Let's first train on the entire dataset to see what we get
lrModel = lrPipeline.fit(trainingSetDF)

From the Wikipedia article on [Linear Regression](https://en.wikipedia.org/wiki/Linear_regression):
> In statistics, linear regression is an approach for modeling the relationship between a scalar dependent variable \\( y \\) and one or more explanatory variables (or independent variables) denoted \\(X\\). In linear regression, the relationships are modeled using linear predictor functions whose unknown model parameters are estimated from the data. Such models are called linear models.

Linear regression has many practical uses. Most applications fall into one of the following two broad categories:
  - If the goal is prediction, or forecasting, or error reduction, linear regression can be used to fit a predictive model to an observed data set of \\(y\\) and \\(X\\) values. After developing such a model, if an additional value of \\(X\\) is then given without its accompanying value of \\(y\\), the fitted model can be used to make a prediction of the value of \\(y\\).
  - Given a variable \\(y\\) and a number of variables \\( X_1 \\), ..., \\( X_p \\) that may be related to \\(y\\), linear regression analysis can be applied to quantify the strength of the relationship between \\(y\\) and the \\( X_j\\), to assess which \\( X_j \\) may have no relationship with \\(y\\) at all, and to identify which subsets of the \\( X_j \\) contain redundant information about \\(y\\).

We are interested in both uses, as we would like to predict power output as a function of the input variables, and we would like to know which input variables are weakly or strongly correlated with power output.

Since Linear Regression is simply a Line of best fit over the data that minimizes the square of the error, given multiple input dimensions we can express each predictor as a line function of the form:

\\[ y = a + b x_1 + b x_2 + b x_i ... \\]

where \\(a\\) is the intercept and the \\(b\\) are the coefficients.

To express the coefficients of that line we can retrieve the Estimator stage from the PipelineModel and express the weights and the intercept for the function.

### Exercise 6(d)

Run the next cell. Ensure that you understand what's going on.

In [58]:
# The intercept is as follows:
intercept = lrModel.stages[1].intercept

# The coefficents (i.e., weights) are as follows:
weights = lrModel.stages[1].coefficients

# Create a list of the column names (without PE)
featuresNoLabel = [col for col in datasetDF.columns if col != "PE"]

# Merge the weights and labels
coefficents = zip(weights, featuresNoLabel)

# Now let's sort the coefficients from greatest absolute weight most to the least absolute weight
coefficents.sort(key=lambda tup: abs(tup[0]), reverse=True)

equation = "y = {intercept}".format(intercept=intercept)
variables = []
for x in coefficents:
    weight = abs(x[0])
    name = x[1]
    symbol = "+" if (x[0] > 0) else "-"
    equation += (" {} ({} * {})".format(symbol, weight, name))

# Finally here is our equation
print("Linear Regression Equation: " + equation)

Linear Regression Equation: y = 436.422672237 - (1.91608232805 * AT) - (0.255230167561 * V) - (0.147726340553 * RH) + (0.0792423513038 * AP)

Recall **Part 4: Visualize Your Data** when we visualized each predictor against Power Output using a Scatter Plot, does the final equation seems logical given those visualizations?

### Exercise 6(e)

Now let's see what our predictions look like given this model. We apply our Linear Regression model to the 15% of the data that we split from the input dataset. The output of the model will be a predicted Power Output column named "Prediction_PE".

- Run the next cell
- Scroll through the resulting table and notice how the values in the Power Output (PE) column compare to the corresponding values in the predicted Power Output (Predicted_PE) column

In [60]:
# Apply our LR model to the test data and predict power output
predictionsAndLabelsDF = lrModel.transform(testSetDF).select("AT", "V", "AP", "RH", "PE", "Prediction_PE")

display(predictionsAndLabelsDF)

AT,V,AP,RH,PE,Prediction_PE
1.81,39.42,1026.92,76.97,490.55,492.6874782987985
3.31,39.42,1024.05,84.31,487.19,488.5625529314632
3.51,35.47,1017.53,86.56,489.07,488.35118033576407
3.69,38.44,1016.74,82.87,490.78,487.6779630621048
3.92,41.31,999.22,95.26,487.35,483.2028910277776
4.47,35.19,1018.78,92.68,487.43,485.85040328888863
4.62,38.44,1016.09,73.14,486.84,487.2491777483213
4.69,39.42,1024.58,79.35,486.34,486.7076257961105
4.73,41.31,999.77,93.44,486.6,481.97959362491986
4.81,42.85,1013.96,86.94,479.8,483.57088852898096


From a visual inspection of the predictions, we can see that they are close to the actual values.

However, we would like a scientific measure of how well the Linear Regression model is performing in accurately predicting values. To perform this measurement, we can use an evaluation metric such as [Root Mean Squared Error](https://en.wikipedia.org/wiki/Root-mean-square_deviation) (RMSE) to validate our Linear Regression model.

RSME is defined as follows: \\( RMSE = \sqrt{\frac{\sum_{i = 1}^{n} (x_i - y_i)^2}{n}}\\) where \\(y_i\\) is the observed value and \\(x_i\\) is the predicted value

RMSE is a frequently used measure of the differences between values predicted by a model or an estimator and the values actually observed. The lower the RMSE, the better our model.

Spark ML Pipeline provides several regression analysis metrics, including [RegressionEvaluator()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.evaluation.RegressionEvaluator).

After we create an instance of [RegressionEvaluator](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.evaluation.RegressionEvaluator), we set the label column name to "PE" and set the prediction column name to "Predicted_PE". We then invoke the evaluator on the predictions.

### Exercise 6(f)

Run the next cell and ensure that you understand what's going on.

In [62]:
# Now let's compute an evaluation metric for our test dataset
from pyspark.ml.evaluation import RegressionEvaluator

# Create an RMSE evaluator using the label and predicted columns
regEval = RegressionEvaluator(predictionCol="Prediction_PE", labelCol="PE", metricName="rmse")

# Run the evaluator on the DataFrame
rmse = regEval.evaluate(predictionsAndLabelsDF)

print("Root Mean Squared Error: %.2f" % rmse)

Root Mean Squared Error: 4.60

Another useful statistical evaluation metric is the coefficient of determination, denoted \\(R^2\\) or \\(r^2\\) and pronounced "R squared". It is a number that indicates the proportion of the variance in the dependent variable that is predictable from the independent variable and it provides a measure of how well observed outcomes are replicated by the model, based on the proportion of total variation of outcomes explained by the model. The coefficient of determination ranges from 0 to 1 (closer to 1), and the higher the value, the better our model.

To compute \\(r^2\\), we invoke the evaluator with  `regEval.metricName: "r2"`

### Exercise 6(g)

Run the next cell and ensure that you understand what's going on.

In [64]:
# Now let's compute another evaluation metric for our test dataset
r2 = regEval.evaluate(predictionsAndLabelsDF, {regEval.metricName: "r2"})

print("r2: {0:.2f}".format(r2))

r2: 0.93

Generally, assuming a Gaussian distribution of errors, a good model will have 68% of predictions within 1 RMSE and 95% within 2 RMSE of the actual value (see http://statweb.stanford.edu/~susan/courses/s60/split/node60.html).

Let's examine the predictions and see if a RMSE of 4.60 meets this criteria.

We create a new DataFrame using [selectExpr()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame.selectExpr) to project a set of SQL expressions, and register the DataFrame as a SQL table using [registerTempTable()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame.registerTempTable).

### Exercise 6(h)

Run the next cell and ensure that you understand what's going on.

In [66]:
# First we remove the table if it already exists
sqlContext.sql("DROP TABLE IF EXISTS Plant_RMSE_Evaluation")
dbutils.fs.rm("dbfs:/user/hive/warehouse/Plant_RMSE_Evaluation", True)

# Next we calculate the residual error and divide it by the RMSE
predictionsAndLabelsDF.selectExpr("PE", "Prediction_PE", "PE - Prediction_PE Residual_Error", "(PE - Prediction_PE) / {} Within_RSME".format(rmse)).registerTempTable("Plant_RMSE_Evaluation")

We can use SQL to explore the `Plant_RMSE_Evaluation` table. First let's look at at the table using a SQL SELECT statement.

### Exercise 6(i)

Run the next cell and ensure that you understand what's going on.

In [68]:
%sql
SELECT * from Plant_RMSE_Evaluation

PE,Prediction_PE,Residual_Error,Within_RSME
490.55,492.6874782987985,-2.1374782987984986,-0.46456588718117325
487.19,488.5625529314632,-1.37255293146319,-0.29831473408022086
489.07,488.35118033576407,0.718819664235923,0.15623040253870446
490.78,487.6779630621048,3.10203693789515,0.6742059289828064
487.35,483.2028910277776,4.147108972222441,0.9013449914324225
487.43,485.85040328888863,1.5795967111113782,0.3433142445929923
486.84,487.2491777483213,-0.40917774832132636,-0.08893190811366067
486.34,486.7076257961105,-0.3676257961105307,-0.07990088330570412
486.6,481.97959362491986,4.620406375080165,1.0042128558606789
479.8,483.57088852898096,-3.7708885289809473,-0.8195761219714207


Now we can display the RMSE as a Histogram.

### Exercise 6(j)

Perform the following steps:

- Run the following cell
- Click on the drop down next to the "Bar chart" icon a select "Histogram" to turn the table into a Histogram plot

<img src="http://spark-mooc.github.io/web-assets/images/cs110x/change-plot-type-histogram.png" style="border: 1px solid #999999"/>


- Click on "Plot Options..."
- In the "All Fields:" box, click on "&lt;id&gt;" and drag it into the "Keys:" box
- Change the "Aggregation" to "COUNT"

<img src="http://spark-mooc.github.io/web-assets/images/cs110x/customize-plot-histogram.png" style="border: 1px solid #999999"/>

- Apply your changes by clicking the Apply button
- Increase the size of the graph by clicking and dragging the size control

Notice that the histogram clearly shows that the RMSE is centered around 0 with the vast majority of the error within 2 RMSEs.

In [70]:
%sql
-- Now we can display the RMSE as a Histogram
SELECT Within_RSME  from Plant_RMSE_Evaluation

Within_RSME
-0.46456588718117325
-0.29831473408022086
0.15623040253870446
0.6742059289828064
0.9013449914324225
0.3433142445929923
-0.08893190811366067
-0.07990088330570412
1.0042128558606789
-0.8195761219714207


Using a complex SQL SELECT statement, we can count the number of predictions within + or - 1.0 and + or - 2.0 and then display the results as a pie chart.

### Exercise 6(k)

Perform the following steps:

  - Run the following cell
  - Click on the drop down next to the "Bar chart" icon a select "Pie" to turn the table into a Pie Chart plot
  - Increase the size of the graph by clicking and dragging the size control

In [72]:
%sql
SELECT case 
       when Within_RSME <= 1.0 AND Within_RSME >= -1.0 then 1
       when Within_RSME <= 2.0 AND Within_RSME >= -2.0 then 2 
       else 3
       end RSME_Multiple, COUNT(*) AS count
FROM Plant_RMSE_Evaluation
GROUP BY RSME_Multiple

RSME_Multiple,count
1,4938
3,221
2,2007


From the pie chart, we can see that 69% of our test data predictions are within 1 RMSE of the actual values, and 97% (69% + 28%) of our test data predictions are within 2 RMSE. So the model is pretty decent. Let's see if we can tune the model to improve it further.

##Part 7: Tuning and Evaluation

Now that we have a model with all of the data let's try to make a better model by tuning over several parameters. The process of tuning a model is known as [Model Selection](https://spark.apache.org/docs/1.6.2/ml-tuning.html#model-selection-aka-hyperparameter-tuning) or [Hyperparameter Tuning](https://spark.apache.org/docs/1.6.2/ml-tuning.html#model-selection-aka-hyperparameter-tuning), and Spark ML Pipeline makes the tuning process very simple and easy.

An important task in ML is model selection, or using data to find the best model or parameters for a given task. This is also called tuning. Tuning may be done for individual Estimators such as [LinearRegression](https://spark.apache.org/docs/1.6.2/ml-classification-regression.html#linear-regression), or for entire Pipelines which include multiple algorithms, featurization, and other steps. Users can tune an entire Pipeline at once, rather than tuning each element in the Pipeline separately.

Spark ML Pipeline supports model selection using tools such as [CrossValidator](https://spark.apache.org/docs/1.6.2/ml-tuning.html#cross-validation), which requires the following items:
  - [Estimator](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.Estimator): algorithm or Pipeline to tune
  - [Set of ParamMaps](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.tuning.ParamGridBuilder): parameters to choose from, sometimes called a _parameter grid_ to search over
  - [Evaluator](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.evaluation.Evaluator): metric to measure how well a fitted Model does on held-out test data

At a high level, model selection tools such as [CrossValidator](https://spark.apache.org/docs/1.6.2/ml-tuning.html#cross-validation) work as follows:
  - They split the input data into separate training and test datasets.
  - For each (training, test) pair, they iterate through the set of ParamMaps:
    - For each [ParamMap](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.tuning.ParamGridBuilder), they fit the [Estimator](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.Estimator) using those parameters, get the fitted Model, and evaluate the Model?s performance using the [Evaluator](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.evaluation.Evaluator).
  - They select the Model produced by the best-performing set of parameters.

The [Evaluator](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.evaluation.Evaluator) can be a [RegressionEvaluator](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.evaluation.RegressionEvaluator) for regression problems. To help construct the parameter grid, users can use the [ParamGridBuilder](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.tuning.ParamGridBuilder) utility.

Note that cross-validation over a grid of parameters is expensive. For example, in the next cell, the parameter grid has 10 values for [lr.regParam](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.regression.LinearRegression.regParam), and [CrossValidator](https://spark.apache.org/docs/1.6.2/ml-tuning.html#cross-validation) uses 3 folds. This multiplies out to (10 x 3) = 30 different models being trained. In realistic settings, it can be common to try many more parameters (e.g., multiple values for multiple parameters) and use more folds (_k_ = 3 and _k_ = 10 are common). In other words, using [CrossValidator](https://spark.apache.org/docs/1.6.2/ml-tuning.html#cross-validation) can be very expensive. However, it is also a well-established method for choosing parameters which is more statistically sound than heuristic hand-tuning.

We perform the following steps:
  - Create a [CrossValidator](https://spark.apache.org/docs/1.6.2/ml-tuning.html#cross-validation) using the Pipeline and [RegressionEvaluator](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.evaluation.RegressionEvaluator) that we created earlier, and set the number of folds to 3
  - Create a list of 10 regularization parameters
  - Use [ParamGridBuilder](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.tuning.ParamGridBuilder) to build a parameter grid with the regularization parameters and add the grid to the [CrossValidator](https://spark.apache.org/docs/1.6.2/ml-tuning.html#cross-validation)
  - Run the [CrossValidator](https://spark.apache.org/docs/1.6.2/ml-tuning.html#cross-validation) to find the parameters that yield the best model (i.e., lowest RMSE) and return the best model.

### Exercise 7(a)

Run the next cell. _Note that it will take some time to run the [CrossValidator](https://spark.apache.org/docs/1.6.2/ml-tuning.html#cross-validation) as it will run almost 200 Spark jobs_

In [75]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# We can reuse the RegressionEvaluator, regEval, to judge the model based on the best Root Mean Squared Error
# Let's create our CrossValidator with 3 fold cross validation
crossval = CrossValidator(estimator=lrPipeline, evaluator=regEval, numFolds=3)

# Let's tune over our regularization parameter from 0.05 to 0.50
regParam = [x / 200.0 for x in range(1, 11)]

# We'll create a paramter grid using the ParamGridBuilder, and add the grid to the CrossValidator
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, regParam)
             .build())
crossval.setEstimatorParamMaps(paramGrid)

# Now let's find and return the best model
cvModel = crossval.fit(trainingSetDF).bestModel

Now that we have tuned our Linear Regression model, let's see what the new RMSE and \\(r^2\\) values are versus our intial model.

### Exercise 7(b)

Complete and run the next cell.

In [77]:
# TODO: Replace <FILL_IN> with the appropriate code.
# Now let's use cvModel to compute an evaluation metric for our test dataset: testSetDF
predictionsAndLabelsDF = cvModel.transform(testSetDF).select("AT", "V", "AP", "RH", "PE", "Prediction_PE")

# Run the previously created RMSE evaluator, regEval, on the predictionsAndLabelsDF DataFrame
rmseNew = regEval.evaluate(predictionsAndLabelsDF)

# Now let's compute the r2 evaluation metric for our test dataset
r2New = regEval.evaluate(predictionsAndLabelsDF, {regEval.metricName: 'r2'})

print("Original Root Mean Squared Error: {0:2.2f}".format(rmse))
print("New Root Mean Squared Error: {0:2.2f}".format(rmseNew))
print("Old r2: {0:2.2f}".format(r2))
print("New r2: {0:2.2f}".format(r2New))

Original Root Mean Squared Error: 4.60
New Root Mean Squared Error: 4.59
Old r2: 0.93
New r2: 0.93

In [78]:
# TEST
Test.assertEquals(round(rmse, 2), 4.60, "Incorrect value for rmse")
Test.assertEquals(round(rmseNew, 2), 4.59, "Incorrect value for rmseNew")
Test.assertEquals(round(r2, 2), 0.93, "Incorrect value for r2")
Test.assertEquals(round(r2New, 2), 0.93, "Incorrect value for r2New")

1 test passed.
1 test passed.
1 test passed.
1 test passed.

So tuned linear regression model improves a little bit over our initial model. Let's look at the regularization parameter that the [CrossValidator](https://spark.apache.org/docs/1.6.2/ml-tuning.html#cross-validation) has selected.

Recall that the orginal regularization parameter we used was 0.015.

**NOTE**: The ML Python API currently doesn't provide a way to query the regularization parameter, so we cheat, by "reaching through" to the JVM version of the API.

In [80]:
print("Regularization parameter of the best model: {0:.2f}".format(cvModel.stages[-1]._java_obj.parent().getRegParam()))

Regularization parameter of the best model: 0.01

Given that the only linearly correlated variable is Temperature, it makes sense try another Machine Learning method such as [Decision Tree](https://en.wikipedia.org/wiki/Decision_tree_learning) to handle non-linear data and see if we can improve our model.

[Decision Tree Learning](https://en.wikipedia.org/wiki/Decision_tree_learning) uses a [Decision Tree](https://en.wikipedia.org/wiki/Decision_tree) as a predictive model which maps observations about an item to conclusions about the item's target value. It is one of the predictive modelling approaches used in statistics, data mining and machine learning. Decision trees where the target variable can take continuous values (typically real numbers) are called regression trees.

Spark ML Pipeline provides [DecisionTreeRegressor()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.regression.DecisionTreeRegressor) as an implementation of [Decision Tree Learning](https://en.wikipedia.org/wiki/Decision_tree_learning).

The cell below is based on the [Spark ML Pipeline API for Decision Tree Regressor](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.regression.DecisionTreeRegressor).

### Exercise 7(c)

- Read the [Decision Tree Regressor](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.regression.DecisionTreeRegressor) documentation
- In the next cell, create a [DecisionTreeRegressor()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.regression.DecisionTreeRegressor)

- The next step is to set the parameters for the method (we do this for you):
  - Set the name of the prediction column to "Prediction_PE"
  - Set the name of the features column to "features"
  - Set the maximum number of bins to 100

- Create the [ML Pipeline](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.Pipeline) and set the stages to the Vectorizer we created earlier and [DecisionTreeRegressor()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.regression.DecisionTreeRegressor) learner we just created.

In [82]:
# TODO: Replace <FILL_IN> with the appropriate code.
from pyspark.ml.regression import DecisionTreeRegressor

# Create a DecisionTreeRegressor
dt = DecisionTreeRegressor()

dt.setLabelCol("PE")\
  .setPredictionCol("Prediction_PE")\
  .setFeaturesCol("features")\
  .setMaxBins(100)

# Create a Pipeline
dtPipeline = Pipeline()

# Set the stages of the Pipeline
dtPipeline.setStages([vectorizer, dt])

Out[ 57 ]: Pipeline_4911ba577b0df64f6c7a

In [83]:
# TEST

Test.assertEqualsHashed(str(dtPipeline.getStages()[0].__class__.__name__), '4617be70bcf475326c0b07400b97b13457cc4949', "Incorrect pipeline stage 0")
Test.assertEqualsHashed(str(dtPipeline.getStages()[1].__class__.__name__), '46b18f257cf2f778d0d3b6e30ccc7b3398d7846a', "Incorrect pipeline stage 1")

1 test passed.
1 test passed.

Instead guessing what parameters to use, we will use [Model Selection](https://spark.apache.org/docs/1.6.2/ml-tuning.html#model-selection-aka-hyperparameter-tuning) or [Hyperparameter Tuning](https://spark.apache.org/docs/1.6.2/ml-tuning.html#model-selection-aka-hyperparameter-tuning) to create the best model.

We can reuse the exiting [CrossValidator](https://spark.apache.org/docs/1.6.2/ml-tuning.html#cross-validation) by replacing the Estimator with our new `dtPipeline` (the number of folds remains 3).

### Exercise 7(d)

- Use [ParamGridBuilder](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.tuning.ParamGridBuilder) to build a parameter grid with the parameter `dt.maxDepth` and a list of the values 2 and 3, and add the grid to the [CrossValidator](https://spark.apache.org/docs/1.6.2/ml-tuning.html#cross-validation)
- Run the [CrossValidator](https://spark.apache.org/docs/1.6.2/ml-tuning.html#cross-validation) to find the parameters that yield the best model (i.e., lowest RMSE) and return the best model.

_Note that it will take some time to run the [CrossValidator](https://spark.apache.org/docs/1.6.2/ml-tuning.html#cross-validation) as it will run almost 50 Spark jobs_

In [85]:
# TODO: Replace <FILL_IN> with the appropriate code.
# Let's just reuse our CrossValidator with the new dtPipeline,  RegressionEvaluator regEval, and 3 fold cross validation
crossval.setEstimator(dtPipeline)

# Let's tune over our dt.maxDepth parameter on the values 2 and 3, create a paramter grid using the ParamGridBuilder
paramGrid = ParamGridBuilder().addGrid(dt.maxDepth, [2, 3]).build()

# Add the grid to the CrossValidator
crossval.setEstimatorParamMaps(paramGrid)

# Now let's find and return the best model
dtModel = crossval.fit(trainingSetDF).bestModel

In [86]:
# TEST
Test.assertEqualsHashed(str(dtModel.stages[0].__class__.__name__), '4617be70bcf475326c0b07400b97b13457cc4949', "Incorrect pipeline stage 0")
Test.assertEqualsHashed(str(dtModel.stages[1].__class__.__name__), 'a2bf7b0c1a0fb9ad35650d0478ad51a9b880befa', "Incorrect pipeline stage 1")

1 test passed.
1 test passed.

### Exercise 7(e)

Now let's see how our tuned DecisionTreeRegressor model's RMSE and \\(r^2\\) values compare to our tuned LinearRegression model.

Complete and run the next cell.

In [88]:
# TODO: Replace <FILL_IN> with the appropriate code.

# Now let's use dtModel to compute an evaluation metric for our test dataset: testSetDF
resultsDF = dtModel.transform(testSetDF).select("AT", "V", "AP", "RH", "PE", "Prediction_PE")

# Run the previously created RMSE evaluator, regEval, on the resultsDF DataFrame
rmseDT = regEval.evaluate(resultsDF)

# Now let's compute the r2 evaluation metric for our test dataset
r2DT = regEval.evaluate(resultsDF, {regEval.metricName:'r2'})

print("LR Root Mean Squared Error: {0:.2f}".format(rmseNew))
print("DT Root Mean Squared Error: {0:.2f}".format(rmseDT))
print("LR r2: {0:.2f}".format(r2New))
print("DT r2: {0:.2f}".format(r2DT))

LR Root Mean Squared Error: 4.59
DT Root Mean Squared Error: 5.16
LR r2: 0.93
DT r2: 0.91

In [89]:
# TEST
Test.assertEquals(round(rmseDT, 2), 5.16, "Incorrect value for rmseDT")
Test.assertEquals(round(r2DT, 2), 0.91, "Incorrect value for r2DT")

1 test passed.
1 test passed.

The line below will pull the Decision Tree model from the Pipeline as display it as an if-then-else string. Again, we have to "reach through" to the JVM API to make this one work.

**TODO**: Run the next cell

In [91]:
print dtModel.stages[-1]._java_obj.toDebugString()

DecisionTreeRegressionModel (uid=DecisionTreeRegressor_46b79a9a6e6690332f6d) of depth 3 with 15 nodes
 If (feature 0 <= 17.71)
 If (feature 0 <= 11.815000000000001)
 If (feature 0 <= 8.765)
 Predict: 483.6650545561782
 Else (feature 0 > 8.765)
 Predict: 475.9447534705601
 Else (feature 0 > 11.815000000000001)
 If (feature 0 <= 14.315000000000001)
 Predict: 469.01933317456536
 Else (feature 0 > 14.315000000000001)
 Predict: 462.42835516739444
 Else (feature 0 > 17.71)
 If (feature 0 <= 22.735)
 If (feature 1 <= 45.935)
 Predict: 458.3270182291667
 Else (feature 1 > 45.935)
 Predict: 449.31361516034997
 Else (feature 0 > 22.735)
 If (feature 1 <= 66.11)
 Predict: 442.9999369686309
 Else (feature 1 > 66.11)
 Predict: 434.8198369053841

So our DecisionTree has slightly worse RMSE than our LinearRegression model (LR: 4.59 vs DT: 5.19). Maybe we can try an [Ensemble Learning](https://en.wikipedia.org/wiki/Ensemble_learning) method such as [Gradient-Boosted Decision Trees](https://en.wikipedia.org/wiki/Gradient_boosting) to see if we can strengthen our model by using an ensemble of weaker trees with weighting to reduce the error in our model.

[Random forests](https://en.wikipedia.org/wiki/Random_forest) or random decision tree forests are an ensemble learning method for regression that operate by constructing a multitude of decision trees at training time and outputting the class that is the mean prediction (regression) of the individual trees. Random decision forests correct for decision trees' habit of overfitting to their training set.

Spark ML Pipeline provides [RandomForestRegressor()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.regression.RandomForestRegressor) as an implementation of [Random forests](https://en.wikipedia.org/wiki/Random_forest).

The cell below is based on the [Spark ML Pipeline API for Random Forest Regressor](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.regression.RandomForestRegressor).

### Exercise 7(f)

- Read the [Random Forest Regressor](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.regression.RandomForestRegressor) documentation
- In the next cell, create a [RandomForestRegressor()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.regression.RandomForestRegressor)
- The next step is to set the parameters for the method (we do this for you):
  - Set the name of the prediction column to "Prediction_PE"
  - Set the name of the features column to "features"
  - Set the random number generator seed to 190088121L
  - Set the maximum depth to 8
  - Set the number of trees to 25
- Create the [ML Pipeline](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.Pipeline) and set the stages to the Vectorizer we created earlier and [RandomForestRegressor()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.regression.RandomForestRegressor) learner we just created.

In [93]:
# TODO: Replace <FILL_IN> with the appropriate code.

from pyspark.ml.regression import RandomForestRegressor

# Create a RandomForestRegressor
rf = RandomForestRegressor()

rf.setLabelCol("PE")\
  .setPredictionCol("Prediction_PE")\
  .setFeaturesCol("features")\
  .setSeed(190088121L)\
  .setMaxDepth(8)\
  .setNumTrees(25)

# Create a Pipeline
rfPipeline = Pipeline()

# Set the stages of the Pipeline
rfPipeline.setStages([vectorizer, rf])

Out[ 65 ]: Pipeline_48e198da64ee06b936c4

In [94]:
# TEST
Test.assertEqualsHashed(rfPipeline.getStages()[0].__class__.__name__, '4617be70bcf475326c0b07400b97b13457cc4949', "Stage 0 of pipeline is not correct")
Test.assertEqualsHashed(rfPipeline.getStages()[1].__class__.__name__, 'ecdcce2d075f00c97a6d2a2b8b1f66de322e57d2', "Stage 1 of pipeline is not correct")

1 test passed.
1 test passed.

As with Decision Trees, instead guessing what parameters to use, we will use [Model Selection](https://spark.apache.org/docs/1.6.2/ml-tuning.html#model-selection-aka-hyperparameter-tuning) or [Hyperparameter Tuning](https://spark.apache.org/docs/1.6.2/ml-tuning.html#model-selection-aka-hyperparameter-tuning) to create the best model.

We can reuse the exiting [CrossValidator](https://spark.apache.org/docs/1.6.2/ml-tuning.html#cross-validation) by replacing the Estimator with our new `rfPipeline` (the number of folds remains 3).

### Exercise 7(g)

- Use [ParamGridBuilder](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.tuning.ParamGridBuilder) to build a parameter grid with the parameter `rf.maxBins` and a list of the values 50 and 100, and add the grid to the [CrossValidator](https://spark.apache.org/docs/1.6.2/ml-tuning.html#cross-validation)
- Run the [CrossValidator](https://spark.apache.org/docs/1.6.2/ml-tuning.html#cross-validation) to find the parameters that yield the best model (i.e., lowest RMSE) and return the best model.

_Note that it will take some time to run the [CrossValidator](https://spark.apache.org/docs/1.6.2/ml-tuning.html#cross-validation) as it will run almost 100 Spark jobs, and each job takes longer to run than the prior CrossValidator runs._

In [96]:
# TODO: Replace <FILL_IN> with the appropriate code.
# Let's just reuse our CrossValidator with the new rfPipeline,  RegressionEvaluator regEval, and 3 fold cross validation
crossval.setEstimator(rfPipeline)

# Let's tune over our rf.maxBins parameter on the values 50 and 100, create a paramter grid using the ParamGridBuilder
paramGrid = ParamGridBuilder().addGrid(rf.maxBins, [50, 100]).build()

# Add the grid to the CrossValidator
crossval.setEstimatorParamMaps(paramGrid)

# Now let's find and return the best model
rfModel = crossval.fit(trainingSetDF).bestModel

In [97]:
# TEST
Test.assertEqualsHashed(rfModel.stages[0].__class__, 'f0c3b910468d87808e019409e7ae5e587d6aca3d', 'rfModel has incorrect stage 0')
Test.assertEqualsHashed(rfModel.stages[1].__class__, '0ed43512ea7e35ebeebeed3ddac0186248999a87', 'rfModel has incorrect stage 1')

1 test passed.
1 test passed.

### Exercise 7(h)

Now let's see how our tuned RandomForestRegressor model's RMSE and \\(r^2\\) values compare to our tuned LinearRegression and tuned DecisionTreeRegressor models.

** TODO**: Complete and run the next cell.

In [99]:
# TODO: Replace <FILL_IN> with the appropriate code.

# Now let's use rfModel to compute an evaluation metric for our test dataset: testSetDF
resultsDF = rfModel.transform(testSetDF).select("AT", "V", "AP", "RH", "PE", "Prediction_PE")

# Run the previously created RMSE evaluator, regEval, on the resultsDF DataFrame
rmseRF = regEval.evaluate(resultsDF)

# Now let's compute the r2 evaluation metric for our test dataset
r2RF = regEval.evaluate(resultsDF, {regEval.metricName:'r2'})

print("LR Root Mean Squared Error: {0:.2f}".format(rmseNew))
print("DT Root Mean Squared Error: {0:.2f}".format(rmseDT))
print("RF Root Mean Squared Error: {0:.2f}".format(rmseRF))
print("LR r2: {0:.2f}".format(r2New))
print("DT r2: {0:.2f}".format(r2DT))
print("RF r2: {0:.2f}".format(r2RF))

LR Root Mean Squared Error: 4.59
DT Root Mean Squared Error: 5.16
RF Root Mean Squared Error: 3.48
LR r2: 0.93
DT r2: 0.91
RF r2: 0.96

In [100]:
# TEST
Test.assertEquals(round(rmseRF, 2), 3.48, "Incorrect value for rmseRF")
Test.assertEquals(round(r2RF, 2), 0.96, "Incorrect value for r2RF")

1 test passed.
1 test passed.

Note that the Decision Tree and Random Forest values for `r2` are identical. However, the RMSE for the Random Forest model is better.

The line below will pull the Random Forest model from the Pipeline as display it as an if-then-else string.

**ToDo**: Run the next cell

In [103]:
print rfModel.stages[-1]._java_obj.toDebugString()

RandomForestRegressionModel (uid=RandomForestRegressor_4dddb99e35b7b8f1797f) with 25 trees
 Tree 0 (weight 1.0):
 If (feature 0 <= 17.795)
 If (feature 0 <= 11.955)
 If (feature 3 <= 95.445)
 If (feature 0 <= 8.625)
 If (feature 0 <= 6.905)
 If (feature 3 <= 67.86500000000001)
 If (feature 1 <= 39.6)
 If (feature 1 <= 35.82)
 Predict: 483.12000000000006
 Else (feature 1 > 35.82)
 Predict: 484.39756756756753
 Else (feature 1 > 39.6)
 If (feature 3 <= 63.245000000000005)
 Predict: 493.80891891891895
 Else (feature 3 > 63.245000000000005)
 Predict: 494.91799999999995
 Else (feature 3 > 67.86500000000001)
 If (feature 3 <= 86.375)
 If (feature 0 <= 5.135)
 Predict: 488.24222772277227
 Else (feature 0 > 5.135)
 Predict: 485.865703125
 Else (feature 3 > 86.375)
 If (feature 2 <= 1021.635)
 Predict: 485.5148846153847
 Else (feature 2 > 1021.635)
 Predict: 482.9366336633655
 Else (feature 0 > 6.905)
 If (feature 2 <= 1022.895)
 If (feature 0 <= 7.845)
 If (feature 2 <= 1019.815)
 Predict: 483.6029268292683
 Else (feature 2 > 1019.815)
 Predict: 484.66644808743183
 Else (feature 0 > 7.845)
 If (feature 1 <= 44.81)
 Predict: 481.9199142367067
 Else (feature 1 > 44.81)
 Predict: 473.0579999999958
 Else (feature 2 > 1022.895)
 If (feature 3 <= 90.94)
 If (feature 3 <= 74.045)
 Predict: 477.7346153846154
 Else (feature 3 > 74.045)
 Predict: 481.62923497267764
 Else (feature 3 > 90.94)
 If (feature 0 <= 7.425000000000001)
 Predict: 466.16
 Else (feature 0 > 7.425000000000001)
 Predict: 475.3890476190477
 Else (feature 0 > 8.625)
 If (feature 0 <= 10.754999999999999)
 If (feature 3 <= 92.87)
 If (feature 1 <= 45.004999999999995)
 If (feature 2 <= 1028.3600000000001)
 Predict: 478.16880054520664
 Else (feature 2 > 1028.3600000000001)
 Predict: 476.2188811188814
 Else (feature 1 > 45.004999999999995)
 If (feature 3 <= 74.045)
 Predict: 473.09
 Else (feature 3 > 74.045)
 Predict: 469.82514285714296
 Else (feature 3 > 92.87)
 If (feature 2 <= 1018.515)
 If (feature 2 <= 1013.395)
 Predict: 472.196
 Else (feature 2 > 1013.395)
 Predict: 478.495909090909
 Else (feature 2 > 1018.515)
 If (feature 2 <= 1022.1)
 Predict: 468.5594285714286
 Else (feature 2 > 1022.1)
 Predict: 474.2376190476193
 Else (feature 0 > 10.754999999999999)
 If (feature 2 <= 1024.33)
 If (feature 3 <= 88.505)
 If (feature 1 <= 45.56)
 Predict: 474.3859927470536
 Else (feature 1 > 45.56)
 Predict: 465.74461538461935
 Else (feature 3 > 88.505)
 If (feature 1 <= 40.629999999999995)
 Predict: 474.82283582089553
 Else (feature 1 > 40.629999999999995)
 Predict: 470.0158241758242
 Else (feature 2 > 1024.33)
 If (feature 1 <= 40.760000000000005)
 If (feature 1 <= 40.04)
 Predict: 468.31999999999994
 Else (feature 1 > 40.04)
 Predict: 485.6806060606059
 Else (feature 1 > 40.760000000000005)
 If (feature 1 <= 40.82)
 Predict: 475.7654545454545
 Else (feature 1 > 40.82)
 Predict: 470.15428571428583
 Else (feature 3 > 95.445)
 If (feature 2 <= 1012.495)
 If (feature 0 <= 8.225000000000001)
 If (feature 0 <= 5.135)
 If (feature 1 <= 41.33)
 If (feature 1 <= 39.93)
 Predict: 490.56208333333325
 Else (feature 1 > 39.93)
 Predict: 489.36800000000005
 Else (feature 1 > 41.33)
 If (feature 3 <= 99.19)
 Predict: 481.88
 Else (feature 3 > 99.19)
 Predict: 482.3579999999999
 Else (feature 0 > 5.135)
 If (feature 0 <= 5.845)
 Predict: 480.2715384615385
 Else (feature 0 > 5.845)
 If (feature 1 <= 39.41)
 Predict: 491.49
 Else (feature 1 > 39.41)
 Predict: 483.1744444444444
 Else (feature 0 > 8.225000000000001)
 If (feature 1 <= 39.144999999999996)
 If (feature 0 <= 11.434999999999999)
 If (feature 2 <= 1009.695)
 Predict: 472.0672580645162
 Else (feature 2 > 1009.695)
 Predict: 473.89866666666666
 Else (feature 0 > 11.434999999999999)
 If (feature 2 <= 1009.815)
 Predict: 460.7
 Else (feature 2 > 1009.815)
 Predict: 470.55
 Else (feature 1 > 39.144999999999996)
 If (feature 2 <= 1011.535)
 If (feature 0 <= 9.055)
 Predict: 476.24799999999993
 Else (feature 0 > 9.055)
 Predict: 473.4650833333333
 Els

### Conclusion

Wow! So our best model is in fact our Random Forest tree model which uses an ensemble of 25 Trees with a depth of 8 to construct a better model than the single decision tree.